In [9]:
import pandas as pd
import os
import json
from functools import singledispatch

In [10]:
date = "2022-01-04"

In [11]:
STATE_NAMES = {
  'AP': 'Andhra Pradesh',
  'AR': 'Arunachal Pradesh',
  'AS': 'Assam',
  'BR': 'Bihar',
  'CT': 'Chhattisgarh',
  'GA': 'Goa',
  'GJ': 'Gujarat',
  'HR': 'Haryana',
  'HP': 'Himachal Pradesh',
  'JH': 'Jharkhand',
  'KA': 'Karnataka',
  'KL': 'Kerala',
  'MP': 'Madhya Pradesh',
  'MH': 'Maharashtra',
  'MN': 'Manipur',
  'ML': 'Meghalaya',
  'MZ': 'Mizoram',
  'NL': 'Nagaland',
  'OR': 'Odisha',
  'PB': 'Punjab',
  'RJ': 'Rajasthan',
  'SK': 'Sikkim',
  'TN': 'Tamil Nadu',
  'TG': 'Telangana',
  'TR': 'Tripura',
  'UT': 'Uttarakhand',
  'UP': 'Uttar Pradesh',
  'WB': 'West Bengal',
  'AN': 'Andaman and Nicobar Islands',
  'CH': 'Chandigarh',
  'DN': 'Dadra and Nagar Haveli and Daman and Diu',
  'DL': 'Delhi',
  'JK': 'Jammu and Kashmir',
  'LA': 'Ladakh',
  'LD': 'Lakshadweep',
  'PY': 'Puducherry',
  'TT': 'India',
 # [UNASSIGNED_STATE_CODE]: 'Unassigned',
}

In [12]:
data_min_json={}
for k,v in STATE_NAMES.items():
    data_min_json[k]={"districts":{},"delta":{},"delta7":{},"delta21_14":{},"meta":{},"total":{}}

In [13]:
def number_generation(df,col_name,col_value,field):
    value=df.loc[df[col_name]==col_value,field]
    value.reset_index(inplace=True,drop=True)
    if value.isna().all():
        value = "null"
    try:
        value = int(value[0])
    except:
        # raise
        value = value[0]
    return value

In [14]:
import json
import traceback
from datetime import datetime
import time

def addLogging(logDict:dict):
    loggingsFile = '../log.json'

    with open(loggingsFile) as f:
        data = json.load(f)
        #data = []

    data.append(logDict)

    with open(loggingsFile, 'w') as f:
        json.dump(data, f)

def currentTimeUTC(date):
    # return time.mktime(datetime.strptime(datetime.now().strftime('%d/%m/%Y'),'%d/%m/%Y').timetuple())
    return int(time.mktime(datetime.strptime(date,'%Y-%m-%d').timetuple()))
    #return datetime.now().strftime('%d/%m/%Y %H:%M:%S')

def updateJSONLog(stateName,ttValue,stateValue,date):
    addLogging({
      "update": stateName + ":\n State totals for confirmed cases reported on https://www.mygov.in/covid-19 ("+str(ttValue)+") different from district total for confirmed cases reported in state government bulletin ("+str(stateValue)+")",
      "timestamp": currentTimeUTC(date)
   })

    # State totals for confirmed cases reported on https://www.mygov.in/covid-19 (xxx) different from district total for confirmed cases reported in state government bulletin (yyy)
# try:
#     print(5/0)
# except ZeroDivisionError:
#     fullTraceback = str(traceback.format_exc())
#     addLogging({'timestamp': currentTimeUTC(), 'level': 'error', 'traceback': fullTraceback})

In [15]:
for k,v in data_min_json.items():
    print(k)
    df = pd.read_csv("../RAWCSV/"+date+"/"+k+"_final.csv")
    df_tt = pd.read_csv("../RAWCSV/"+date+"/TT_final.csv")
    # df = pd.read_csv('datamin.csv',header=[1])
    for district in df["District"]:
        try:
            district_dict={district:{"delta":{"confirmed":number_generation(df,"District",district,"deltaConfirmedForDistrict"),
                                              "recovered":number_generation(df,"District",district,"deltaRecoveredForDistrict"),
                                              "deceased":number_generation(df,"District",district,"deltaDeceasedForDistrict"),
                                              "vaccinated1":number_generation(df,"District",district,"deltaVaccinated1ForDistrict"),
                                              "vaccinated2":number_generation(df,"District",district,"deltaVaccinated2ForDistrict")
                                             },
                                    # },#}
            # district_dict={district:{
                                     "delta7":{
                                    "confirmed": number_generation(df,"District",district,"7DmaConfirmedForDistrict"),
                                    "deceased": number_generation(df,"District",district,"7DmaDeceasedForDistrict"),
                                    "recovered": number_generation(df,"District",district,"7DmaRecoveredForDistrict"),
                                    "vaccinated1": number_generation(df,"District",district,"7DmaVaccinated1ForDistrict"),
                                    "vaccinated2": number_generation(df,"District",district,"7DmaVaccinated2ForDistrict")
                                   },
            #                }
            #               }
                           "meta":{"population": number_generation(df,"District",district,"districtPopulation"),
                                 "tested": {
                                  "last_updated": number_generation(df,"State/UTCode",k,'last_updated'),
                                  "source": number_generation(df,"State/UTCode",k,'tested_source_state'),
                                 },
                                 # "notes": number_generation(df,"State/UTCode",k,'notesForDistrict')
                           },
                            "total":{
                               "confirmed": number_generation(df,"District",district,'cumulativeConfirmedNumberForDistrict'),
                                 "deceased": number_generation(df,"District",district,'cumulativeDeceasedNumberForDistrict'),
                                 "recovered": number_generation(df,"District",district,'cumulativeRecoveredNumberForDistrict'),
                                 # "tested": number_generation(df,"State/UTCode",k,'cumulativeTestedNumberForDistrict'),
                                 "vaccinated1": number_generation(df,"District",district,'cumulativeVaccinated1NumberForDistrict'),
                                "vaccinated2": number_generation(df,"District",district,'cumulativeVaccinated2NumberForDistrict')
                           }}
                          }
            
            
        except KeyError as e:
            print(district,e)
            pass
        else:
            data_min_json[k]["districts"].update(district_dict)
            
            
    print("State:"+str(number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState')))
    print("TT   :"+str(number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict')))
    if k != "TT":
        if number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState') != number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict'):
            updateJSONLog(STATE_NAMES[k],number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict'),number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState'),date)
            # pass
    if k != "TT":
        data_min_json[k]["delta"]["confirmed"]=number_generation(df_tt,"District",STATE_NAMES[k],'deltaConfirmedForDistrict')
        data_min_json[k]["delta"]["deceased"]=number_generation(df_tt,"District",STATE_NAMES[k],'deltaDeceasedForDistrict')
        data_min_json[k]["delta"]["recovered"]=number_generation(df_tt,"District",STATE_NAMES[k],'deltaRecoveredForDistrict')
    else:
        data_min_json[k]["delta"]["confirmed"]=number_generation(df,"State/UTCode",k,'deltaConfirmedForState')
        data_min_json[k]["delta"]["deceased"]=number_generation(df,"State/UTCode",k,'deltaDeceasedForState')
        data_min_json[k]["delta"]["recovered"]=number_generation(df,"State/UTCode",k,'deltaRecoveredForState')
    
    data_min_json[k]["delta"]["vaccinated1"]=number_generation(df,"State/UTCode",k,'deltaVaccinated1ForState')
    data_min_json[k]["delta"]["vaccinated2"]=number_generation(df,"State/UTCode",k,'deltaVaccinated2ForState')
    data_min_json[k]["delta"]["tested"]=number_generation(df,"State/UTCode",k,'deltaTestedForState')
    # if k != "TT":
    #     data_min_json[k]["delta"]["tested"]=number_generation(df,"State/UTCode",k,'deltaTestedForState')
    # else:
    #     data_min_json[k]["delta"]["tested"]=0
    
    data_min_json[k]["delta7"]["confirmed"]=number_generation(df,"State/UTCode",k,'7DmaConfirmedForState')
    data_min_json[k]["delta7"]["deceased"]=number_generation(df,"State/UTCode",k,'7DmaDeceasedForState')
    data_min_json[k]["delta7"]["recovered"]=number_generation(df,"State/UTCode",k,'7DmaRecoveredForState')
    data_min_json[k]["delta7"]["vaccinated1"]=number_generation(df,"State/UTCode",k,'7DmaVaccinated1ForState')
    data_min_json[k]["delta7"]["vaccinated2"]=number_generation(df,"State/UTCode",k,'7DmaVaccinated2ForState')
    data_min_json[k]["delta7"]["tested"]=number_generation(df,"State/UTCode",k,'7DmaTestedForState')
    data_min_json[k]["delta21_14"]={"confirmed":number_generation(df,"State/UTCode",k,'delta21_14confirmedForState')}
    
    data_min_json[k]["meta"]={"date":number_generation(df,"State/UTCode",k,'Date'),
                             "last_updated":number_generation(df,"State/UTCode",k,'last_updated'),
                              "notes":number_generation(df,"State/UTCode",k,'notesForState'),
                              "population":number_generation(df,"State/UTCode",k,'statePopulation'),
                              "tested":{"date":number_generation(df,"State/UTCode",k,'last_updated'),
                                        "source":number_generation(df,"State/UTCode",k,'tested_source_state')
                              }
                             }
    if k != "TT":
        data_min_json[k]["total"]["confirmed"] = number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict')
        data_min_json[k]["total"]["deceased"] = number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeDeceasedNumberForDistrict')
        data_min_json[k]["total"]["recovered"] = number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeRecoveredNumberForDistrict')
    else:
        data_min_json[k]["total"]["confirmed"] = number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState')
        data_min_json[k]["total"]["deceased"] = number_generation(df,"State/UTCode",k,'cumulativeDeceasedNumberForState')
        data_min_json[k]["total"]["recovered"] = number_generation(df,"State/UTCode",k,'cumulativeRecoveredNumberForState')
    
    
    data_min_json[k]["total"]["tested"] = number_generation(df,"State/UTCode",k,'cumulativeTestedNumberForState')
    data_min_json[k]["total"]["vaccinated1"] = number_generation(df,"State/UTCode",k,'cumulativeVaccinated1NumberForState')
    data_min_json[k]["total"]["vaccinated2"] = number_generation(df,"State/UTCode",k,'cumulativeVaccinated2NumberForState')

AP
State:2077942
TT   :2077942
AR
State:55375
TT   :55375
AS
State:621897
TT   :621897
BR
State:728766
TT   :728766
CT
State:1010513
TT   :1010513
GA
State:182793
TT   :182793
GJ
State:837131
TT   :837293
HR
State:776842
TT   :776842
HP
State:229413
TT   :229413
JH
State:357965
TT   :357965
KA
State:3013326
TT   :3013326
KL
State:5254974
TT   :5258614
MP
State:794769
TT   :794769
MH
State:6730494
TT   :6730494
MN
State:125878
TT   :125878
ML
State:84881
TT   :84881
MZ
State:142691
TT   :142691
NL
State:32210
TT   :32210
OR
State:1056660
TT   :1056660
PB
State:606927
TT   :606927
RJ
State:958570
TT   :958570
SK
State:32530
TT   :32530
TN
State:2755587
TT   :2755587
TG
State:684023
TT   :684023
TR
State:85211
TT   :85214
UT
State:345963
TT   :345963
UP
State:1714096
TT   :1714096
WB
State:1664301
TT   :1664301
AN
State:7799
TT   :7799
CH
State:66264
TT   :66264
DN
State:10701
TT   :10701
DL
State:1463701
TT   :1463701
JK
State:342001
TT   :342001
LA
State:22321
TT   :22321
LD
State:10419

In [16]:
@singledispatch
def remove_null_bool(ob):
    return ob

@remove_null_bool.register(list)
def _process_list(ob):
    return [remove_null_bool(v) for v in ob]

@remove_null_bool.register(dict)
def _process_list(ob):
    return {k: remove_null_bool(v) for k, v in ob.items()
            if v is not None and v is not 0 and v is not 'n' and v is not {}}

with open('data.min_swi.json', 'w') as json_file:
    json.dump(remove_null_bool(data_min_json), json_file)

# Stop here

In [42]:
import requests, json

In [43]:
url = requests.get("https://data.covid19india.org/v4/min/data.min.json")
text = url.text

In [44]:
C19I_data = json.loads(text)

In [35]:
compare_json_data(C19I_data['RJ'],remove_null_bool(data_min_json))

False

In [36]:
print('Compare JSON result is: {0}'.format(
		compare_json_data(C19I_data['RJ'],remove_null_bool(data_min_json))
	))

Compare JSON result is: False


In [21]:
!pip install jsondiff

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for jsondiff, since package 'wheel' is not installed.
    Running setup.py install for jsondiff: started
    Running setup.py install for jsondiff: finished with status 'done'


In [11]:
import jsondiff

In [45]:
res = jsondiff.diff(C19I_data,remove_null_bool(data_min_json))

In [47]:
res

{'AN': {'delta21_14': {replace: {}},
  'delta7': {'confirmed': 7649,
   'recovered': 7516,
   'deceased': 129,
   'vaccinated': 494119,
   delete: ['tested', 'vaccinated1', 'vaccinated2']},
  'districts': {'Nicobars': {replace: {'delta': {}}},
   'North and Middle Andaman': {replace: {'delta': {}}},
   'South Andaman': {replace: {'delta': {}}},
   'Unknown': {replace: {'delta': {'recoverd': 1}}}},
  'meta': {'last_updated': '2021-10-31 13:23:20.429082',
   'tested': {'date': '2021-10-31 13:23:20.429082', delete: ['source']},
   'Date': '2021-10-30',
   delete: ['date']},
  'total': {'vaccinated2': 200121, delete: ['tested']},
  'delta': {'recovered': 1, 'vaccinated': 1495}},
 'AP': {'delta': {replace: {'vaccinated': 420158}},
  'delta21_14': {replace: {}},
  'delta7': {'confirmed': 2064023,
   'deceased': 14359,
   'recovered': 2044892,
   'vaccinated': 53305600,
   delete: ['tested', 'vaccinated1', 'vaccinated2']},
  'districts': {'Anantapur': {'delta': {replace: {}},
    delete: ['de

In [46]:
remove_null_bool(data_min_json)

{'AP': {'districts': {'Anantapur': {'delta': {}},
   'Chittoor': {'delta': {}},
   'East Godavari': {'delta': {}},
   'Guntur': {'delta': {}},
   'Y.S.R. Kadapa': {'delta': {}},
   'Krishna': {'delta': {}},
   'Kurnool': {'delta': {}},
   'S.P.S. Nellore': {'delta': {}},
   'Prakasam': {'delta': {}},
   'Srikakulam': {'delta': {}},
   'Visakhapatnam': {'delta': {}},
   'Vizianagaram': {'delta': {}},
   'West Godavari': {'delta': {}}},
  'delta': {'vaccinated': 420158},
  'delta7': {'confirmed': 2064023,
   'deceased': 14359,
   'recovered': 2044892,
   'vaccinated': 53305600},
  'delta21_14': {},
  'meta': {'Date': '30-10-2021',
   'last_updated': '35:50.3',
   'population': 52221000,
   'tested': {'date': '35:50.3'}},
  'total': {'confirmed': 2063170,
   'deceased': 14369,
   'recovered': 2044152,
   'vaccinated1': 32955532,
   'vaccinated2': 20350068}},
 'AR': {'districts': {'Anjaw': {'delta': {}},
   'Changlang': {'delta': {'comfirmed': 2}},
   'East Kameng': {'delta': {'deceased': 

In [40]:
42374177 + 19518213

61892390